## 顾客消费分析
有一份customer-orders.csv文件，文件中的每一行记录了一位顾客消费的数据，3列分别表示

* 顾客id
* 消费id
* 消费费用

In [1]:
# 我们只关心用户id和消费费用，所以选出了第1和第3列
import findspark
findspark.init()
from pyspark import SparkConf, SparkContext

conf = SparkConf().setMaster('local').setAppName('CustomerAnalysis')
sc = SparkContext(conf = conf)

def parseLine(line):
    fields = line.split(',')
    return (int(fields[0]), float(fields[2]))

lines = sc.textFile('./data/customer-orders.csv')
rdd = lines.map(parseLine)

### 取出记录前3行

In [3]:
rdd.take(3)

[(44, 37.19), (35, 65.89), (2, 40.64)]

### 统计每位顾客的消费总额

In [4]:
totalByCustomer = rdd.reduceByKey(lambda x, y: x+y)
totalByCustomer.take(3)

[(44, 4756.8899999999985), (35, 5155.419999999999), (2, 5994.59)]

### 取出消费最高的5位顾客
tips：可以查询课程和速查表中的sortBy和sortByKey函数

In [5]:
totalByCustomer.sortBy(lambda x:x[1], ascending=False).take(5)

[(68, 6375.449999999997),
 (73, 6206.199999999999),
 (39, 6193.109999999999),
 (54, 6065.389999999999),
 (71, 5995.660000000003)]

### 消费超过5000的顾客有多少位
tips：可以查询课程和速查表中的filter和count函数

In [6]:
totalByCustomer.filter(lambda x:x[1]>5000).count()

49

### 消费超过5200的顾客，平均消费是多少

In [7]:
totalByCustomer.filter(lambda x:x[1]>5200).map(lambda x:x[1]).mean()

5546.428333333334

### 消费最高的20名顾客比最低的20位顾客，平均消费高多少

In [8]:
(sum(totalByCustomer.map(lambda x:x[1]).top(20))- sum(totalByCustomer.map(lambda x:x[1]).takeOrdered(20)))/20

1465.9774999999995

In [9]:
# 停止sparkContext
sc.stop()

## 电影评分分析
有一份u.data文件，文件中的每一行记录了一条电影打分记录，4列用tab分隔开分别表示

* 顾客id
* 电影id
* 电影打分
* 时间戳

In [10]:
import findspark
findspark.init()
# 读取文件
from pyspark import SparkConf, SparkContext

conf = SparkConf().setMaster('local').setAppName('RatingsHistogram')
sc = SparkContext(conf = conf)

lines = sc.textFile('./data/u.data')
lines.take(3)

['196\t242\t3\t881250949', '186\t302\t3\t891717742', '22\t377\t1\t878887116']

In [13]:
# 我们只保留前3列
ratings = lines.map(lambda x: x.split()[2])
ratings.take(3)

['3', '3', '1']

### 统计不同打分的数量

In [14]:
result = ratings.countByValue()
result

defaultdict(int, {'3': 27145, '1': 6110, '2': 11370, '4': 34174, '5': 21201})

### 统计影评数量最多的3部电影
tips：有点像我们的词频统计问题，大家应该知道怎么做了吧？

In [17]:
movies = lines.map(lambda x: (int(x.split()[1]), 1))
movies.take(3)

[(242, 1), (302, 1), (377, 1)]

In [18]:
movieCounts = movies.reduceByKey(lambda x, y: x + y)
movieCounts.sortBy(lambda x:x[1], ascending=False).take(3)

[(50, 583), (258, 509), (100, 508)]

### 找到影评数量最多的3位user

In [19]:
users = lines.map(lambda x: (int(x.split()[0]), 1))
userCounts = users.reduceByKey(lambda x, y: x + y)
userCounts.sortBy(lambda x:x[1], ascending=False).take(3)

[(405, 737), (655, 685), (13, 636)]

### 统计每个user的平均影评电影数量

In [20]:
userCounts.map(lambda x: x[1]).mean()

106.04453870625665

### 统计全局的平均分

In [21]:
lines.map(lambda x: int(x.split()[2])).mean()

3.529859999999947

### 高分电影(超过3分)的平均分

In [35]:
rdd = lines.map(lambda x: (int(x.split()[1]), int(x.split()[2])))
rdd.take(3)

[(242, 3), (302, 3), (377, 1)]

In [38]:
rdd = rdd.filter(lambda x: x[1]>3)
rdd.take(3)

[(474, 4), (465, 5), (1014, 5)]

In [39]:
result =rdd.map(lambda x:x[1]).mean()
result

4.38286230248308

In [40]:
lines.map(lambda x: (int(x.split()[1]), int(x.split()[2])))\
        .filter(lambda x: x[1]>3)\
        .map(lambda x:x[1])\
        .mean()

4.38286230248308

In [41]:
# newRDD = rdd.filter(lambda x:x[2] > '3').map(lambda x:(x[2],1)).reduceByKey(lambda x, y: x + y)

# result_list = newRDD.map(lambda x:int(x[0]) * x[1]).collect()

# sum(result_list) / rdd.filter(lambda x:x[2] > '3').count()

In [42]:
# 停止sparkContext
sc.stop()